In [22]:
# importing necessary packages


import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [23]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []

In [24]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
        
# using an if here because tags can repeat    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [30]:
# convert the target labels into a model understandable form.
# “LabelEncoder()” function provided by scikit-learn
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [31]:
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [32]:
vocab_size = 1000 # the number of words to keep, based on frequency
# why we limit vocab size?
# we don’t want every unique token in our vocabulary. If it doesn’t appear at 
# least twice then might just be a spelling mistake or a 
# word we can’t learn anything about it if it doesn’t appear that often.

# we might think that low frequency words (especially hapax legomena) 
# don't tell us very much useful information! And practically, this is often true; you don't 
# benefit from increasing the vocab size past a certain point, 
# but you will continue to incur increasing memory and performance costs associated with a larger input representation.

embedding_dim = 16
max_len = 20
oov_token = "<OOV>" # oov_token is a value for “out of token”

# Tokenization is a method to segregate a particular text into small chunks or tokens. 
# Here the tokens or chunks can be anything from words to characters, even subwords.

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)

# fitting the tokenizer on training sentences
tokenizer.fit_on_texts(training_sentences)

# the index of the words in tokenizer. this is a dict
word_index = tokenizer.word_index

# texts_to_sequences method in tokenizer class helps in converting tokens of text corpus into a sequence of integers.
sequences = tokenizer.texts_to_sequences(training_sentences)

# The “pad_sequences” method is used to make all the training text sequences into the same size.
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [37]:
# model training using sequential model of keras

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
__________________________________________________

In [38]:
# fitting the model on the training labels and training sequences
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - ETA: 0s - loss: 2.0787 - accuracy: 0.09 - 0s 3ms/step - loss: 2.0792 - accuracy: 0.0909
Epoch 2/500
2/2 [==============================] - ETA: 0s - loss: 2.0776 - accuracy: 0.15 - 0s 3ms/step - loss: 2.0778 - accuracy: 0.1515
Epoch 3/500
2/2 [==============================] - ETA: 0s - loss: 2.0771 - accuracy: 0.21 - 0s 3ms/step - loss: 2.0772 - accuracy: 0.2121
Epoch 4/500
2/2 [==============================] - ETA: 0s - loss: 2.0767 - accuracy: 0.25 - 0s 2ms/step - loss: 2.0769 - accuracy: 0.2424
Epoch 5/500
2/2 [==============================] - ETA: 0s - loss: 2.0772 - accuracy: 0.21 - 0s 3ms/step - loss: 2.0767 - accuracy: 0.2424
Epoch 6/500
2/2 [==============================] - ETA: 0s - loss: 2.0761 - accuracy: 0.21 - 0s 2ms/step - loss: 2.0764 - accuracy: 0.2121
Epoch 7/500
2/2 [==============================] - ETA: 0s - loss: 2.0758 - accuracy: 0.21 - 0s 3ms/step - loss: 2.0757 - accuracy: 0.2121
Epoch 8/500
2/2 [==========

2/2 [==============================] - ETA: 0s - loss: 2.0225 - accuracy: 0.46 - 0s 3ms/step - loss: 2.0239 - accuracy: 0.4545
Epoch 60/500
2/2 [==============================] - ETA: 0s - loss: 2.0219 - accuracy: 0.37 - 0s 3ms/step - loss: 2.0223 - accuracy: 0.3636
Epoch 61/500
2/2 [==============================] - ETA: 0s - loss: 2.0201 - accuracy: 0.34 - 0s 3ms/step - loss: 2.0201 - accuracy: 0.3636
Epoch 62/500
2/2 [==============================] - ETA: 0s - loss: 2.0180 - accuracy: 0.37 - 0s 2ms/step - loss: 2.0175 - accuracy: 0.3636
Epoch 63/500
2/2 [==============================] - ETA: 0s - loss: 2.0144 - accuracy: 0.37 - 0s 4ms/step - loss: 2.0150 - accuracy: 0.3636
Epoch 64/500
2/2 [==============================] - ETA: 0s - loss: 2.0086 - accuracy: 0.37 - 0s 2ms/step - loss: 2.0126 - accuracy: 0.3636
Epoch 65/500
2/2 [==============================] - ETA: 0s - loss: 2.0089 - accuracy: 0.37 - 0s 2ms/step - loss: 2.0100 - accuracy: 0.3636
Epoch 66/500
2/2 [===============

2/2 [==============================] - ETA: 0s - loss: 1.8188 - accuracy: 0.43 - 0s 3ms/step - loss: 1.8195 - accuracy: 0.4545
Epoch 118/500
2/2 [==============================] - ETA: 0s - loss: 1.8142 - accuracy: 0.43 - 0s 4ms/step - loss: 1.8133 - accuracy: 0.4545
Epoch 119/500
2/2 [==============================] - ETA: 0s - loss: 1.8006 - accuracy: 0.50 - 0s 3ms/step - loss: 1.8061 - accuracy: 0.4848
Epoch 120/500
2/2 [==============================] - ETA: 0s - loss: 1.8107 - accuracy: 0.46 - 0s 3ms/step - loss: 1.7975 - accuracy: 0.4848
Epoch 121/500
2/2 [==============================] - ETA: 0s - loss: 1.8008 - accuracy: 0.46 - 0s 3ms/step - loss: 1.7862 - accuracy: 0.4848
Epoch 122/500
2/2 [==============================] - ETA: 0s - loss: 1.7920 - accuracy: 0.46 - 0s 3ms/step - loss: 1.7762 - accuracy: 0.4848
Epoch 123/500
2/2 [==============================] - ETA: 0s - loss: 1.7645 - accuracy: 0.53 - 0s 3ms/step - loss: 1.7702 - accuracy: 0.5152
Epoch 124/500
2/2 [========

2/2 [==============================] - ETA: 0s - loss: 1.4963 - accuracy: 0.53 - 0s 3ms/step - loss: 1.4640 - accuracy: 0.5455
Epoch 176/500
2/2 [==============================] - ETA: 0s - loss: 1.4888 - accuracy: 0.50 - 0s 2ms/step - loss: 1.4573 - accuracy: 0.5152
Epoch 177/500
2/2 [==============================] - ETA: 0s - loss: 1.4516 - accuracy: 0.53 - 0s 2ms/step - loss: 1.4536 - accuracy: 0.5152
Epoch 178/500
2/2 [==============================] - ETA: 0s - loss: 1.4556 - accuracy: 0.50 - 0s 3ms/step - loss: 1.4498 - accuracy: 0.5152
Epoch 179/500
2/2 [==============================] - ETA: 0s - loss: 1.4455 - accuracy: 0.50 - 0s 3ms/step - loss: 1.4461 - accuracy: 0.5152
Epoch 180/500
2/2 [==============================] - ETA: 0s - loss: 1.4256 - accuracy: 0.53 - 0s 3ms/step - loss: 1.4415 - accuracy: 0.5152
Epoch 181/500
2/2 [==============================] - ETA: 0s - loss: 1.4357 - accuracy: 0.50 - 0s 2ms/step - loss: 1.4359 - accuracy: 0.5152
Epoch 182/500
2/2 [========

2/2 [==============================] - ETA: 0s - loss: 1.2498 - accuracy: 0.50 - 0s 3ms/step - loss: 1.2408 - accuracy: 0.5152
Epoch 234/500
2/2 [==============================] - ETA: 0s - loss: 1.2440 - accuracy: 0.50 - 0s 3ms/step - loss: 1.2376 - accuracy: 0.5152
Epoch 235/500
2/2 [==============================] - ETA: 0s - loss: 1.2635 - accuracy: 0.50 - 0s 2ms/step - loss: 1.2352 - accuracy: 0.5152
Epoch 236/500
2/2 [==============================] - ETA: 0s - loss: 1.2662 - accuracy: 0.50 - 0s 3ms/step - loss: 1.2333 - accuracy: 0.5152
Epoch 237/500
2/2 [==============================] - ETA: 0s - loss: 1.2654 - accuracy: 0.50 - 0s 2ms/step - loss: 1.2322 - accuracy: 0.5152
Epoch 238/500
2/2 [==============================] - ETA: 0s - loss: 1.2112 - accuracy: 0.53 - 0s 3ms/step - loss: 1.2321 - accuracy: 0.5152
Epoch 239/500
2/2 [==============================] - ETA: 0s - loss: 1.2223 - accuracy: 0.53 - 0s 3ms/step - loss: 1.2318 - accuracy: 0.5152
Epoch 240/500
2/2 [========

2/2 [==============================] - ETA: 0s - loss: 1.0689 - accuracy: 0.53 - 0s 2ms/step - loss: 1.0984 - accuracy: 0.5152
Epoch 292/500
2/2 [==============================] - ETA: 0s - loss: 1.1026 - accuracy: 0.50 - 0s 3ms/step - loss: 1.0995 - accuracy: 0.5152
Epoch 293/500
2/2 [==============================] - ETA: 0s - loss: 1.0666 - accuracy: 0.53 - 0s 2ms/step - loss: 1.0966 - accuracy: 0.5152
Epoch 294/500
2/2 [==============================] - ETA: 0s - loss: 1.1056 - accuracy: 0.50 - 0s 2ms/step - loss: 1.0897 - accuracy: 0.5152
Epoch 295/500
2/2 [==============================] - ETA: 0s - loss: 1.1019 - accuracy: 0.50 - 0s 2ms/step - loss: 1.0814 - accuracy: 0.5152
Epoch 296/500
2/2 [==============================] - ETA: 0s - loss: 1.0713 - accuracy: 0.53 - 0s 2ms/step - loss: 1.0746 - accuracy: 0.5152
Epoch 297/500
2/2 [==============================] - ETA: 0s - loss: 1.0485 - accuracy: 0.53 - 0s 2ms/step - loss: 1.0718 - accuracy: 0.5152
Epoch 298/500
2/2 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
# save all the required files in order to use it at the inference time. 
# we save the trained model, fitted tokenizer object and fitted label encoder object.

# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: hello
ChatBot: Hi there
User: who are you?
ChatBot: I.m Joana, your bot assistant
User: 